# Common utitities for all production alphas

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
if !@isdefined source_name
    source_name = "all/Inference"
    @warn "source_name not defined: using $source_name"
end;

In [ ]:
function get_recommendee_username()
    @assert @isdefined username
    username
end;

In [ ]:
function recommendee_alpha_basepath()
    username = get_recommendee_username()
    mkpath(get_data_path("recommendations/$username/alphas"))
end;

## Reading and writing data

In [ ]:
function get_recommendee_split(content::String, medium::String)
    raw_split(content, medium) = get_raw_recommendee_split(content, medium)
    if content == "explicit"
        df = raw_split("explicit", medium)
    elseif content == "implicit"
        df = cat(raw_split("explicit", medium), raw_split("implicit", medium))
        df.rating .= 1
    elseif content == "ptw"
        df = raw_split("ptw", medium)
    else
        @assert false
    end
    df
end

function get_raw_recommendee_split(content::String, medium::String)
    @assert content in ["explicit", "implicit", "ptw"]
    @assert medium in ALL_MEDIUMS
    username = get_recommendee_username()
    file = get_data_path("recommendations/$username/$medium.$content.jld2")
    JLD2.load(file, "dataset")
end;

In [ ]:
function write_recommendee_alpha(ratings::Vector{Float32}, medium::String, outdir::String)
    @assert medium in ALL_MEDIUMS
    @assert length(ratings) == num_items(medium)
    alpha = RatingsDataset(rating = ratings, medium = medium)
    outdir = mkpath("$(recommendee_alpha_basepath())/$outdir")
    JLD2.save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [ ]:
function read_recommendee_alpha(alpha::String, content::String, medium::String)
    @assert content in ["explicit", "implicit", "ptw", "all"]
    @assert medium in ALL_MEDIUMS
    df = JLD2.load("$(recommendee_alpha_basepath())/$alpha/alpha.jld2")["alpha"]
    df = RatingsDataset(
        user = fill(1, num_items(medium)),
        item = 1:num_items(medium),
        rating = df.rating,
        medium = medium,
    )
    if content == "all"
        return df
    else
        mask = get_recommendee_split(content, medium).item
        return RatingsDataset(
            user = df.user[mask],
            item = df.item[mask],
            rating = df.rating[mask],
            medium = medium,
        )
    end
end;

In [ ]:
@memoize get_regression_beta(alphas, task, content, medium, implicit) =
    regress(alphas, task, content, medium, implicit)[2]

function read_recommendee_alpha(
    alphas::Vector{String},
    task::String,
    content::String,
    medium::String,
    implicit::Bool,
)
    β = get_regression_beta(alphas, task, content, medium, implicit)
    ratings = zeros(Float32, num_items(medium))
    for i = 1:length(alphas)
        ratings += β[i] .* read_recommendee_alpha(alphas[i], "all", medium).rating
    end
    if implicit
        @assert length(alphas) + 1 == length(β)
        ratings .+= β[end] * 1 / num_items(medium)
    end

    df = get_recommendee_split(content, medium)
    RatingsDataset(
        user = df.user,
        item = df.item,
        rating = ratings[df.item],
        medium = medium,
    )
end;